# Two-Views RGBD Odometry

ref: http://www.open3d.org/docs/tutorial/Basic/rgbd_odometry.html

An RGBD odometry finds the camera movement between two consecutive RGBD image pairs. The input are two instances of RGBDImage. The output is the motion in the form of a rigid body transformation. Open3D has implemented two RGBD odometries: [Steinbrucker2011] and [Park2017].

In [3]:
# src/Python/Tutorial/Basic/rgbd_odometry.rst

from open3d import *
import numpy as np

## Read camera intrinsic
We first read the camera intrinsic matrix from a json file.

In [4]:
pinhole_camera_intrinsic = read_pinhole_camera_intrinsic(
        "/data/code6/Open3D/build/lib/TestData/camera.json")
print(pinhole_camera_intrinsic.intrinsic_matrix)

[[415.69219382   0.         319.5       ]
 [  0.         415.69219382 239.5       ]
 [  0.           0.           1.        ]]


## Read RGBD image
reads two pairs of RGBD images in the Redwood format

In [6]:
source_color = read_image("/data/code6/Open3D/build/lib/TestData/RGBD/color/00000.jpg")
source_depth = read_image("/data/code6/Open3D/build/lib/TestData/RGBD/depth/00000.png")
target_color = read_image("/data/code6/Open3D/build/lib/TestData/RGBD/color/00001.jpg")
target_depth = read_image("/data/code6/Open3D/build/lib/TestData/RGBD/depth/00001.png")
source_rgbd_image = create_rgbd_image_from_color_and_depth(
        source_color, source_depth);
target_rgbd_image = create_rgbd_image_from_color_and_depth(
        target_color, target_depth);

In [7]:
target_pcd = create_point_cloud_from_rgbd_image(
        target_rgbd_image, pinhole_camera_intrinsic)

In [8]:
option = OdometryOption()
odo_init = np.identity(4) # identity matrix for intial odometry
print(option)

OdometryOption class.
minimum_correspondence_ratio = 0.100000
iteration_number_per_pyramid_level = [ 20, 10, 5, ] 
max_depth_diff = 0.030000
min_depth = 0.000000
max_depth = 4.000000


In [9]:
odo_init

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

## Compute odometry from two RGBD image pairs

In [10]:
# [Steinbrucker2011] odometry method
# minimizes photo consistency of aligned images.
[success_color_term, trans_color_term, info] = compute_rgbd_odometry(
        source_rgbd_image, target_rgbd_image,
        pinhole_camera_intrinsic, odo_init,
        RGBDOdometryJacobianFromColorTerm(), option)

In [11]:
# [Park2017] odometry method
# in addition to photo consistency, it implements constraint for geometry
[success_hybrid_term, trans_hybrid_term, info] = compute_rgbd_odometry(
        source_rgbd_image, target_rgbd_image,
        pinhole_camera_intrinsic, odo_init,
        RGBDOdometryJacobianFromHybridTerm(), option)

In [12]:
if success_color_term:
    print("Using RGB-D Odometry")
    print(trans_color_term)
    source_pcd_color_term = create_point_cloud_from_rgbd_image(
            source_rgbd_image, pinhole_camera_intrinsic)
    source_pcd_color_term.transform(trans_color_term)
    draw_geometries([target_pcd, source_pcd_color_term])

Using RGB-D Odometry
[[ 9.99985161e-01 -2.24233601e-04 -5.44316453e-03 -4.82515882e-04]
 [ 1.46064327e-04  9.99896920e-01 -1.43571495e-02  2.89043658e-02]
 [ 5.44582281e-03  1.43561414e-02  9.99882115e-01  7.88078866e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [15]:
if success_hybrid_term:
    print("Using Hybrid RGB-D Odometry")
    print(trans_hybrid_term)
    source_pcd_hybrid_term = create_point_cloud_from_rgbd_image(
            source_rgbd_image, pinhole_camera_intrinsic)
    source_pcd_hybrid_term.transform(trans_hybrid_term)
    draw_geometries([target_pcd, source_pcd_hybrid_term])

Using Hybrid RGB-D Odometry
[[ 9.99994666e-01 -1.00290832e-03 -3.10826679e-03 -3.75410519e-03]
 [ 9.64494137e-04  9.99923448e-01 -1.23356688e-02  2.54977515e-02]
 [ 3.12040039e-03  1.23326051e-02  9.99919082e-01  1.88139777e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
